In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import datetime
import io
import os
import glob
import sys
from googleads import adwords
from datetime import datetime, timedelta
import hashlib
import uuid

#https://developers.google.com/adwords/api/docs/samples/python/remarketing

def NormalizeAndSHA256(s):
    
    return hashlib.sha256(s.strip().lower()).hexdigest()


output= io.StringIO()

adwords_client = adwords.AdWordsClient.LoadFromStorage('xxx.yaml')

adwords_client.SetClientCustomerId('xxx')

user_list_service = adwords_client.GetService('AdwordsUserListService', 'v201809')

user_list = {
  'xsi_type': 'CrmBasedUserList',
  'name': 'Триггерное взаимодействие #%d' % uuid.uuid4(),
  'description': 'Почты пользователей для триггерного взаимодействия',
  # CRM-based user lists can use a membershipLifeSpan of 10000 to indicate
  # unlimited; otherwise normal values apply.
  'membershipLifeSpan': 30,
  'uploadKeyType': 'CONTACT_INFO'
}

operations = [{
  'operator': 'ADD',
  'operand': user_list
}]

result = user_list_service.mutate(operations)
user_list_id = result['value'][0]['id']


emails = ['customer1@example.com', 'customer2@example.com',
        ' Customer3@example.com ']
members = [{'hashedEmail': NormalizeAndSHA256(email.encode('utf-8'))} for email in emails]


In [ ]:
mutate_members_operation

In [ ]:
#members.append({
#  'addressInfo': {
#      # First and last name must be normalized and hashed.
#      'hashedFirstName': NormalizeAndSHA256('John'.encode('utf-8')),
#      'hashedLastName': NormalizeAndSHA256('Doe'.encode('utf-8')),
#      # Country code and zip code are sent in plaintext.
#      'countryCode': 'US',
#      'zipCode': '10001'
#  }
#})


mutate_members_operation = {
  'operand': {
      'userListId': user_list_id,
      'membersList': members
  },
  'operator': 'ADD'
}

In [ ]:
response = user_list_service.mutateMembers([mutate_members_operation])

In [ ]:
if 'userLists' in response:
    for user_list in response['userLists']:
        print('User list with name "{}" and ID "%{}" was added'.format(user_list['name'], user_list['id']))

#https://ads.google.com/aw/audiences/management?ocid=82065735&euid=237824820&__u=8323372180&uscid=82065735&__c=1792984015&authuser=0